In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# ✅ Wikipedia S&P 500 기업 목록 페이지 URL
URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# ✅ Selenium WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # 브라우저 창 숨김
options.add_argument("--no-sandbox")  # 충돌 방지
options.add_argument("--disable-dev-shm-usage")  # 메모리 문제 해결
options.add_argument("--disable-gpu")  # GPU 가속 비활성화
options.add_argument("user-agent=Mozilla/5.0")

# ✅ WebDriver 실행
driver = webdriver.Chrome(options=options)
driver.get(URL)

# ✅ 페이지 로드 대기
time.sleep(5)

# ✅ 첫 번째 테이블 가져오기
try:
    table = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CLASS_NAME, "wikitable"))
    )

    # ✅ 테이블에서 행 추출
    rows = table.find_elements(By.TAG_NAME, "tr")[1:]  # 첫 번째 행(헤더) 제외

    company_data = []
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        if len(cols) < 2:
            continue  # 데이터가 부족한 경우 스킵

        # ✅ 기본 정보 추출
        ticker = cols[0].text.strip()
        company_name = cols[1].text.strip()

        # ✅ 각 회사의 Wikipedia 페이지 URL 가져오기
        try:
            company_link_element = cols[1].find_element(By.TAG_NAME, "a")
            company_link = company_link_element.get_attribute("href")
        except:
            company_link = "Unknown"

        # ✅ 상세 페이지에서 개요 텍스트 크롤링
        overview_text = "Unknown"
        if company_link != "Unknown":
            driver.execute_script("window.open(arguments[0]);", company_link)
            driver.switch_to.window(driver.window_handles[1])
            time.sleep(3)

            try:
                # ✅ 개요 부분 가져오기
                content_div = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "mw-content-ltr"))
                )

                paragraphs = content_div.find_elements(By.TAG_NAME, "p")

                # ✅ "mw-heading mw-heading2" 이전까지의 텍스트 수집
                overview_list = []
                for p in paragraphs:
                    if p.find_elements(By.CLASS_NAME, "mw-heading2"):
                        break  # 개요 부분 종료
                    overview_list.append(p.text.strip())

                overview_text = "\n".join(overview_list)

            except:
                pass

            # ✅ 새 창 닫고 원래 창으로 복귀
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

        # ✅ 데이터 저장
        company_data.append({
            "ticker": ticker,
            "company_name": company_name,
            "overview": overview_text
        })

        # ✅ 테스트를 위해 5개 기업까지만 크롤링 (전체 크롤링 시 제거)
        if len(company_data) >= 500:
            break

    # ✅ DataFrame 생성 후 저장
    df = pd.DataFrame(company_data)
    df.to_csv("sp500_companies.csv", index=False, encoding="utf-8")
    print("✅ S&P 500 기업 크롤링 완료! ‘sp500_companies.csv’ 파일 저장됨.")

except Exception as e:
    print(f"❌ 오류 발생: {e}")

# ✅ WebDriver 종료
driver.quit()